In [9]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from pathlib import Path
from math import acos, degrees

In [10]:
folder = 'data/'

cap = cv2.VideoCapture('data/video1.mp4')

# cap1 = cv2.VideoCapture('video_dataset\\dataset\\train\\Tadasana\\video1.mp4')
while True:
    success, img = cap.read()
    if not success:
        break
    cv2.imshow("video", img)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [11]:
videos = []
directory_path = Path('data')
for file_path in directory_path.glob('*'):
    print(file_path)
    videos.append(str(file_path))

data\2023-05-15 08-50-17.MP4
data\2023-05-15 09-14-20.MP4
data\2023-05-15 11-17-32.MP4
data\2023-06-06 12-32-06.MP4
data\2023-06-06 12-35-43.MP4


In [12]:
def get_angle(dot1, dot2, dot3):
    if not (dot1 and dot2 and dot3):
        return 0
    a = (dot2.x - dot1.x, dot2.y - dot1.y, dot2.z - dot1.z)
    b = (dot2.x - dot3.x, dot2.y - dot3.y, dot2.z - dot3.z)
    skalar_mul = a[0] * b[0] + a[1] * b[1]
    norm_mul = ((a[0] ** 2 + a[1] ** 2) * (b[0] ** 2 + b[1] ** 2)) ** 0.5
    return degrees(acos(skalar_mul / norm_mul))

In [13]:
def save_frame(path, img):
    cv2.imwrite(path, img)
    print(f"Кадр сохранен в {path}")

In [15]:
def draw_on_video(video, display_size, speed):
    print(video)
    cap = cv2.VideoCapture(video)
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    a = max(frame_width, frame_height) // display_size    
    output_size = (frame_width // a, frame_height // a)
    n_frame = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while True:
            n_frame += 1
            success, frame = cap.read()
            if  n_frame % speed != 0:
                continue
            if cv2.waitKey(1) == ord('q'):
                break
            if not success:
                break
                    
            resized_frame = cv2.resize(frame, output_size, interpolation=cv2.INTER_AREA)
                
            image = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
      
            # Make detection
            results = pose.process(image)
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Render detections
            if results.pose_landmarks:
                
                # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                #                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                #                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                #                          )
                left_angle = get_angle(results.pose_landmarks.landmark[23], 
                                       results.pose_landmarks.landmark[25], 
                                       results.pose_landmarks.landmark[27])
                    
                right_angle = get_angle(results.pose_landmarks.landmark[24], 
                                        results.pose_landmarks.landmark[26], 
                                        results.pose_landmarks.landmark[28])

                if left_angle > 160 and right_angle > 160:
                    # cv2.putText(image, 'good', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    save_frame(f'legs_pics/straight/pic{n_frame}.jpg', image)
                elif left_angle < 120 or right_angle < 120:
                    # cv2.putText(image, 'bad', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    save_frame(f'legs_pics/bent/pic{n_frame}.jpg', image)
                # else:
                    # cv2.putText(image, 'medium', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            
                # cv2.putText(image, 
                #             str(left_angle), 
                #             (int(results.pose_landmarks.landmark[25].x * output_size[0]), int(results.pose_landmarks.landmark[25].y * output_size[1])), 
                #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                
                # cv2.putText(image, 
                #             str(left_angle), 
                #             (int(results.pose_landmarks.landmark[26].x * output_size[0]), int(results.pose_landmarks.landmark[26].y * output_size[1])), 
                #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                
            cv2.imshow("video", image)
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
display_size = 600
speed = 50

for video in videos:
    draw_on_video(video, display_size, speed)

data\2023-05-15 08-50-17.MP4
Кадр сохранен в legs_pics/straight/pic50.jpg
Кадр сохранен в legs_pics/straight/pic100.jpg
Кадр сохранен в legs_pics/straight/pic150.jpg
Кадр сохранен в legs_pics/straight/pic200.jpg
Кадр сохранен в legs_pics/straight/pic250.jpg
Кадр сохранен в legs_pics/straight/pic300.jpg
Кадр сохранен в legs_pics/straight/pic350.jpg
Кадр сохранен в legs_pics/straight/pic500.jpg
Кадр сохранен в legs_pics/straight/pic550.jpg
Кадр сохранен в legs_pics/straight/pic600.jpg
Кадр сохранен в legs_pics/straight/pic650.jpg
Кадр сохранен в legs_pics/straight/pic750.jpg
Кадр сохранен в legs_pics/straight/pic800.jpg
Кадр сохранен в legs_pics/straight/pic850.jpg
Кадр сохранен в legs_pics/straight/pic900.jpg
Кадр сохранен в legs_pics/straight/pic950.jpg
Кадр сохранен в legs_pics/straight/pic1100.jpg
Кадр сохранен в legs_pics/straight/pic1150.jpg
Кадр сохранен в legs_pics/straight/pic1200.jpg
Кадр сохранен в legs_pics/straight/pic1250.jpg
Кадр сохранен в legs_pics/straight/pic1300.jpg
К

In [ ]:
import cv2

# Открытие исходного видео
input_video = 'input_video.mp4'
cap = cv2.VideoCapture(input_video)

# Получение свойств исходного видео
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_size = (frame_width // 2, frame_height // 2)  # Уменьшение размера в 2 раза

# Создание объекта для записи нового видео
output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, output_size)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Изменение размера кадра
    resized_frame = cv2.resize(frame, output_size, interpolation=cv2.INTER_AREA)

    # Запись измененного кадра в новый файл
    out.write(resized_frame)

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()
